# Web Scraping Using Selenium

[Source](https://towardsdatascience.com/web-scraping-using-selenium-python-8a60f4cf40ab)

## What is web scraping?

Web scraping is a technique for extracting information from the internet automatically using a software that simulates human web surfing.

## How is web-scraping useful?

Web scraping helps us extract large volumes of data about customers, products, people, stock markets, etc. It is usually difficult to get this kind of information on a large scale using traditional data collection methods. We can utilize the data collected from a website such as e-commerce portal, social media channels to understand customer behaviors and sentiments, buying patterns, and brand attribute associations which are critical insights for any business.

## Getting Started

Since we have defined our purpose of scraping, let us delve into the nitty-gritty of how to actually do all the fun stuff! Before that below are some of the housekeeping instructions regarding installations of packages.

* Python version: We will be using Python 3.0
* Selenium package: You can install selenium package using the following command

In [ ]:
!pip install selenium

* Chrome driver: Please install the latest version of chromedriver from [here](https://chromedriver.storage.googleapis.com/index.html?path=2.42/).

Please note you need [Google Chrome](https://support.google.com/chrome/answer/95346?co=GENIE.Platform%3DDesktop&hl=en)installed on your machines to work through this illustration.

The first and foremost thing while scraping a website is to understand the structure of the website. We will be scraping [Edmunds.com](https://www.edmunds.com/), a car forum. This website aids people in their car buying decisions. People can post their reviews about different cars in the discussion forums (very similar to how one posts reviews on Amazon). We will be scraping the discussion about [entry level luxury car brands](https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p702).

We will scrape ~5000 comments from different users across multiple pages. We will scrape user id, date of comment and comments and export it into a csv file for any further analysis.

## Let’s begin

We will first import important packages in our Notebook

In [10]:
#Importing packages
import pandas as pd
from selenium import webdriver

Let’s now create a new instance of google chrome. This will help our program open an url in google chrome.

In [11]:
from webdriver_manager.microsoft import EdgeChromiumDriverManager

driver = webdriver.Edge(EdgeChromiumDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Driver [/Users/bk/.wdm/drivers/edgedriver/mac64/94.0.992.50/msedgedriver] found in cache
/var/folders/jd/pq0swyt521jb2424d6fvth840000gn/T/ipykernel_23762/1678680.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(EdgeChromiumDriverManager().install())


Let’s now access google chrome and open our [website](https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p702). By the way, chrome knows that you are accessing it through an automated software!

In [12]:
driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p1')

**Woha! We just opened an url from python notebook.**

So, how does our web page look like?

We will inspect 3 items (user id, date and comment) on our web page and understand how we can extract them.

1. User id: Inspecting the userid, we can see the highlighted text represents the XML code for user id.

![img1](https://miro.medium.com/max/3840/1*JZ9A8VrVdr7GXllaxCfq-w.png)

The XML path (XPath)for the userid is shown below. There is an interesting thing to note here that the XML path contains a comment id, which uniquely denotes each comment on the website. This will be very helpful as we try to recursively scrape multiple comments .

```
//*[@id=”Comment_5561090"]/div/div[2]/div[1]/span[1]/a[2]
```

If we see the XPath in the picture, we will observe that it contains the user id ‘dino001’.

### How do we extract the values inside a XPath?

Selenium has a function called “find_elements_by_xpath”. We will pass our XPath into this function and get a selenium element. Once we have the element, we can extract the text inside our XPath using the ‘text’ function. In our case the text is basically the user id (‘dino001’).

In [14]:
userid_element = driver.find_elements_by_xpath('//*[@id="Comment_5561090"]/div/div[2]/div[1]/span[1]/a[2]')[0]
userid = userid_element.text

/var/folders/jd/pq0swyt521jb2424d6fvth840000gn/T/ipykernel_23762/2211819048.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  userid_element = driver.find_elements_by_xpath('//*[@id="Comment_5561090"]/div/div[2]/div[1]/span[1]/a[2]')[0]


IndexError: list index out of range

2. Comment Date: Similar to the user id, we will now inspect the date when the comment was posted.

![img2](https://miro.medium.com/max/3840/1*GxVX1WtmgjKhuxliygMH8Q.png)

Let’s also see the XPath for the comment date. Again note the unique comment id in the XPath.

```
//*[@id="Comment_5561090"]/div/div[2]/div[2]/span[1]/a/time
```

### So, how do we extract date from the above XPath?

We will again use the function “find_elements_by_xpath” to get the selenium element. Now, if we carefully observe the highlighted text in the picture, we will see that the date is stored inside the ‘title’ attribute. We can access the values inside attributes using the function ‘get_attribute’. We will pass the tag name in this function to get the value inside the same.

```
user_date = driver.find_elements_by_xpath('//*[@id="Comment_5561090"]/div/div[2]/div[2]/span[1]/a/time')[0]
date = user_date.get_attribute('title')
```

3. Comments: Lastly, let’s explore how to extract the comments of each user.

![img3](https://miro.medium.com/max/3840/1*MmLjPWxX6Pn9SV0ia1NajQ.png)

Below is the XPath for the user comment.

```
//*[@id="Comment_5561090"]/div/div[3]/div/div[1]
```

Once again, we have the comment id in our XPath. Similar to the userid we will extract the comment from the above XPath

```python
user_message = driver.find_elements_by_xpath('//*[@id="Comment_5561090"]/div/div[3]/div/div[1]')[0]
comment = user_message.text
```

### We just learnt how to scrape different elements from a web page. Now how to recursively extract these items for 5000 users?

As discussed above, we will use the comment ids, which are unique for a comment to extract different users data. If we see the XPath for the entire comment block, we will see that it has a comment id associated with it.

```
//*[@id="Comment_5561090"]
```

![img4](https://miro.medium.com/max/3840/1*npG4HgblaPtFO7Q3qK69rA.png)

The following code snippet will help us extract all the comment ids on a particular web page. We will again use the function ‘find_elements_by_xpath’ on the above XPath and extract the ids from the ‘id’ attribute.

This code gives us a list of all the comment ids from a particular web page.

In [15]:
ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
comment_ids = []
for i in ids:
    comment_ids.append(i.get_attribute('id'))

/var/folders/jd/pq0swyt521jb2424d6fvth840000gn/T/ipykernel_23762/2346684339.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")


## How to bring all this together?

Now we will bring all the things we have seen so far into one big code, which will recursively help us extract 5000 comments. We can extract user ids, date and comments for each user on a particular web page by looping through all the comment ids we found in the previous code.

Below is the code snippet to extract all comments from a particular web page.

In [17]:
driver = webdriver.Edge(EdgeChromiumDriverManager().install())

comments = pd.DataFrame(columns = ['Date','user_id','comments']) 
comment_ids = []

for page in range(1, 11, 1):
    driver.get(f'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p{page}')
    ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
    for i in ids:
        comment_ids.append(i.get_attribute('id'))
    
    for x in comment_ids:
        #Extract dates from for each user on a page
        user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
        date = user_date.get_attribute('title')

        #Extract user ids from each user on a page
        userid_element = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
        userid = userid_element.text

        #Extract Message for each user on a page
        user_message = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[3]/div/div[1]')[0]
        comment = user_message.text

        #Adding date, userid and comment for each user in a dataframe    
        comments.loc[len(comments)] = [date,userid,comment]

[WDM] - ====== WebDriver manager ======
[WDM] - Driver [/Users/bk/.wdm/drivers/edgedriver/mac64/94.0.992.50/msedgedriver] found in cache
/var/folders/jd/pq0swyt521jb2424d6fvth840000gn/T/ipykernel_23762/3028486432.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(EdgeChromiumDriverManager().install())
/var/folders/jd/pq0swyt521jb2424d6fvth840000gn/T/ipykernel_23762/3028486432.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
/var/folders/jd/pq0swyt521jb2424d6fvth840000gn/T/ipykernel_23762/3028486432.py:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
/var/folders/jd/pq0swyt521jb2424d6fvth840000gn/T/ipykernel_23762/3028486432.py:18: Depreca

IndexError: list index out of range

In [33]:
comments.head()

,Date,user_id,comments
0,"March 25, 2002 5:54AM",merc1,I personally think that with a few tweaks the ...
1,"March 25, 2002 7:06AM",fredvh,I am debating a new purchase and these two are...
2,"March 25, 2002 5:02PM",blueguydotcom,"Great handling, RWD, excellent engine and the ..."
3,"March 25, 2002 11:04PM",hungrywhale,And no manual tranny. That may not matter to y...
4,"March 26, 2002 12:44AM",riez,One beauty of BMW 3 Series is that there are s...
